In [2]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
from tvDatafeed import TvDatafeed, Interval
import multiprocessing
import time
import glob

     |████████████████████████████████| 6.4 MB 6.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 1.8 MB/s 
     |████████████████████████████████| 979 kB 3.4 MB/s 
     |████████████████████████████████| 54 kB 1.1 MB/s 
     |████████████████████████████████| 359 kB

In [14]:
def get_nasq_syms():
  with open("watchlist_NASDAQ.txt", "r") as f:
    lines = f.readlines()
    f.close()
  line = lines[0].split(",")
  symbols = []
  for l in line:
    x = l.split(":")
    symbols.append(x[1])
  return symbols
def download_data_p(tv,sym,exc,tf):
    data =tv.get_hist(sym,exchange=exc, interval=tf, n_bars=50000)
    if data.empty:
      pass
    else:
         try:
           if np.array(data).shape[0] > 15:
             data.to_csv(f"/content/data/{sym}.csv")
         except:
            pass
def download_data(symbols):
  work_with_dir()
  tv = TvDatafeed()
  for sym in symbols:
    print(f'{symbols.index(sym)+1}/{len(symbols)} ',end=" ")
    p = multiprocessing.Process(target=download_data_p, name="dd", args=(tv,sym,"binance",Interval.in_monthly))
    p.start()
    time.sleep(2)
    p.terminate()
    p.join()
def get_tbinances():
  with open("bsymbols.txt","r") as f:
     lines = f.readlines()
  nl = []
  for line in lines:
    nl.append(line.strip())
  return nl
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")   

In [ ]:
symbols = get_tbinances()
print(f"Symbols: {len(symbols)}")
download_data(symbols,)

In [15]:
clmns = list(range(1, 61))
clmns.append("suggestion")
def scaler(row):
    scaler = MinMaxScaler(feature_range=(0,1))
    row = scaler.fit_transform(row)
    return row
def make_y(data,i):
  sugg = "sell"
  if data[i][3] > data[i-1][3]:
      sugg = "buy"
  return sugg
def process(data):
    data = data.dropna()
    row = []
    data.drop("symbol",axis=1,inplace=True)
    data.drop("datetime",axis=1,inplace=True)
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(51, data.shape[0]-1):
        grow = []
        ggggrow = []
        for x in range(1, 51):
            grow.append([data[i-x][0]] )
            ggggrow.append([data[i-x][3] ])
        arr = np.array(grow).flatten()
        arr4 = np.array(ggggrow).flatten()
        sugg = make_y(data,i)
        arr = np.concatenate((arr, arr4), axis=0).reshape(-1,1)
        #arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [12]:
folder_name = extract_data(20)
to_par(folder_name,5)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 

KeyboardInterrupt: ignored

In [ ]:
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=25,batch_size=1024,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"TMSG127_{acr}.h5")

In [ ]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        ggggrow = []
        gggggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][0] - data[i-(1+x)][0]] )
            ggrow.append([data[i-x][1] - data[i-(1+x)][1]] )
            gggrow.append([data[i-x][2] - data[i-(1+x)][2]] )
            ggggrow.append([data[i-x][3] - data[i-(1+x)][3]] )
            gggggrow.append([data[i-x][-1] - data[i-(1+x)][-1]] )

        arr = np.array(grow).flatten()
        arr2 = np.array(ggrow).flatten()
        arr3 = np.array(gggrow).flatten()
        arr4 = np.array(ggggrow).flatten()
        arr5 = np.array(gggggrow).flatten()

        arr = scaler(arr.reshape(-1, 1))
        arr2 = scaler(arr2.reshape(-1, 1))
        arr3 = scaler(arr3.reshape(-1, 1))
        arr4 = scaler(arr4.reshape(-1, 1))
        arr5 = scaler(arr5.reshape(-1, 1))

        arr = np.concatenate((arr, arr2, arr3, arr4,arr5), axis=0).reshape(-1,1)
        return arr
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe),index)
    return f"YF  : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
   tv = TvDatafeed()
   raw_data = process_for_prediction(tv.get_hist(symbol=symbol,exchange=exchange,interval=timeframe,n_bars=35),tindex)
   return  f"TVB : {model.predict(np.array(raw_data).reshape(1,-1))}"
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [ ]:
make_prediction("btc-usd","max","1mo","btcusd","bitstamp",Interval.in_monthly,-1,-2)